In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, GRU, Lambda
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

import tensorflow as tf

import tensorflow_hub as hub

from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
from keras.initializers import Constant

!pip install np_utils
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical

from sklearn.linear_model import Perceptron
from sklearn.multiclass import OneVsRestClassifier

from keras.layers.embeddings import Embedding
from keras.preprocessing import text, sequence


# Import Plotting Libararies
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Model Evaluation Libraries
from sklearn.metrics import classification_report, confusion_matrix

     |████████████████████████████████| 61 kB 3.5 MB/s 
  Created wheel for np-utils: filename=np_utils-0.5.12.1-py3-none-any.whl size=57131 sha256=881bf8d7e0b0128486eff0ced8f82021aef512658c7b7616fbf064dc4f0041c1
  Stored in directory: /root/.cache/pip/wheels/8c/4e/ef/095c24693723c329f4cdc1079861cdbb2487d4b41b2496a4e7
Successfully built np-utils


In [2]:
data = pd.read_csv("/content/sample_data/Sarcasm Dataset (1).csv")

In [3]:
#data = data.dropna()
data = data.fillna(' ')
data = data[['tweet', 'sarcastic']]

X = data['tweet']
Y = data["sarcastic"]

In [4]:
import spacy
df_w2v = data.copy() 
def tokenizer(str):
    nlp = spacy.blank('en')
    doc = nlp.tokenizer(str)
    return [i.text for i in doc]

df_w2v['tweet'] = df_w2v['tweet'].apply(lambda x: x.lower())
df_w2v['tweet'] = df_w2v['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


for idx,row in df_w2v.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 900000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_w2v['tweet'].values)
X = tokenizer.texts_to_sequences(df_w2v['tweet'].values)
X = pad_sequences(X)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle = True, test_size=0.33)

In [6]:
X_train.shape

(2323, 63)

In [7]:
##################################### Word2Vec #################################################################

sentences = [sublist.split() for sublist in data['tweet'].astype(str).values]
words = set([item for sublist in sentences for item in sublist])

import gensim
#Dimension of vectors we are generating
EMBEDDING_DIM = 300

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences = words , size=EMBEDDING_DIM , window = 5 , min_count = 1)

len(w2v_model.wv.vocab)

tokenizer = text.Tokenizer(num_words=6000)
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
x = sequence.pad_sequences(tokenized_train, maxlen = 20)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

def get_weight_matrix(model, vocab):

  vocab_size = len(vocab) + 1

  weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

  for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv.vocab:
      weight_matrix[i] = model[word]
  return weight_matrix

#Getting embedding vectors from word2vec and usings it as weights of non-trainable keras embedding layer
embedding_vectors = get_weight_matrix(w2v_model, tokenizer.word_index)


#Defining Neural Network
model1 = Sequential()
#Non-trainable embeddidng layer
model1.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=63, trainable=True))
#LSTM 
model1.add(LSTM(units=32 , recurrent_dropout = 0.3 , dropout = 0.3))
#model.add(Bidirectional(GRU(units=32 , recurrent_dropout = 0.1 , dropout = 0.1)))
model1.add(Dense(1, activation='sigmoid'))
#model1.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['acc'])
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

del embedding_vectors

model1.summary()

print("Accuracy of the model on Training Data is - " , model1.evaluate(X_train,y_train)[1]*100)
print("Accuracy of the model on Testing Data is - " , model1.evaluate(X_test,y_test)[1]*100)

history1 = model1.fit(X_train, y_train, epochs=2, batch_size=20)

test_prediction1 = model1.predict(X_test)

y_prediction = list()
a=0
b=1
for val in test_prediction1:
  if val < 0.4:
    y_prediction.append(a)
  else:
    y_prediction.append(b)

class_report = classification_report(y_prediction, y_test)
conf_matrix = confusion_matrix(y_prediction , y_test)

print('Results \n')
print(class_report)
print(conf_matrix)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 63, 300)           3410700   
                                                                 
 lstm (LSTM)                 (None, 32)                42624     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 3,453,357
Trainable params: 3,453,357
Non-trainable params: 0
_________________________________________________________________
73/73 [==============================] - 2s 17ms/step - loss: 0.6957 - accuracy: 0.4731
Accuracy of the model on Training Data is -  47.30951488018036
36/36 [==============================] - 1s 16ms/step - loss: 0.6958 - accuracy: 0.4489
Accuracy of the model on Testing Data is -  44.89082992076874
Epoch 1/2

In [15]:
####################################### Glove ###############################################

embedding_dict={}
with open('/content/sample_data/glove.6B.50d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

num_words=len(words)+1
embedding_matrix=np.zeros((num_words,50))

for word,i in tqdm(tokenizer.word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec


X_train,X_val, y_train, y_val = train_test_split(X,Y, test_size=.2, random_state=2)

model2=Sequential()

embedding_layer=Embedding(num_words,50,embeddings_initializer=Constant(embedding_matrix), input_length=63,trainable=False)

model2.add(embedding_layer)

model2.add(tf.keras.layers.LSTM(32))

model2.add(Dense(1, activation='sigmoid'))


model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model2.summary()

print("Accuracy of the model on Training Data is - " , model2.evaluate(X_train,y_train)[1]*100)
print("Accuracy of the model on Testing Data is - " , model2.evaluate(X_test,y_test)[1]*100)


history2 = model2.fit(X_train, y_train, epochs=2, batch_size=20)


test_prediction2 = model2.predict(X_test)

y_prediction = list()
a=0
b=1
for val in test_prediction2:
  if val < 0.4:
    y_prediction.append(a)
  else:
    y_prediction.append(b)

class_report = classification_report(y_prediction, y_test)
conf_matrix = confusion_matrix(y_prediction , y_test)

print('Results \n')
print(class_report)
print(conf_matrix)

100%|██████████| 11368/11368 [00:00<00:00, 431048.38it/s]


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 63, 50)            788450    
                                                                 
 lstm_3 (LSTM)               (None, 32)                10624     
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 799,107
Trainable params: 10,657
Non-trainable params: 788,450
_________________________________________________________________
87/87 [==============================] - 2s 9ms/step - loss: 0.6356 - accuracy: 0.7203
Accuracy of the model on Training Data is -  72.02595472335815
36/36 [==============================] - 0s 9ms/step - loss: 0.6356 - accuracy: 0.7240
Accuracy of the model on Testing Data is -  72.40174412727356
Epoch 1/

In [ ]:
##################################### Fasttext ##############################################



In [10]:
!pip install kaggle

In [11]:
!sudo mkdir ~/.kaggle

!sudo cp /content/sample_data/kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json



mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m -p /content/kaggle

 99% 1.43G/1.44G [00:14<00:00, 105MB/s]
100% 1.44G/1.44G [00:14<00:00, 107MB/s]


In [13]:
!unzip -u '/content/kaggle/fasttext-crawl-300d-2m.zip' -d '/content/sample_data/DL_word2vec_glove_fastext'

Archive:  /content/kaggle/fasttext-crawl-300d-2m.zip
  inflating: /content/sample_data/DL_word2vec_glove_fastext/crawl-300d-2M.vec  


In [14]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback


EMBEDDING_FILE = '/content/sample_data/DL_word2vec_glove_fastext/crawl-300d-2M.vec'

embedding_dict={}
with open(EMBEDDING_FILE ,'r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

num_words=len(words)+1
embedding_matrix=np.zeros((num_words,300))

for word,i in tqdm(tokenizer.word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec


X_train,X_val, y_train, y_val = train_test_split(X,Y, test_size=.2, random_state=2)

model3=Sequential()

embedding_layer=Embedding(num_words,300,weights=[embedding_matrix], input_length=63,trainable=False)

model3.add(embedding_layer)

model3.add(tf.keras.layers.LSTM(32))

model3.add(Dense(1, activation='sigmoid'))


model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model3.summary()

print("Accuracy of the model on Training Data is - " , model3.evaluate(X_train,y_train)[1]*100)
print("Accuracy of the model on Testing Data is - " , model3.evaluate(X_test,y_test)[1]*100)


history3 = model3.fit(X_train, y_train, epochs=2, batch_size=20)


test_prediction3 = model3.predict(X_test)

y_prediction = list()
a=0
b=1
for val in test_prediction3:
  if val < 0.4:
    y_prediction.append(a)
  else:
    y_prediction.append(b)

class_report = classification_report(y_prediction, y_test)
conf_matrix = confusion_matrix(y_prediction , y_test)

print('Results \n')
print(class_report)
print(conf_matrix)

100%|██████████| 11368/11368 [00:00<00:00, 409007.33it/s]


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 63, 300)           4730700   
                                                                 
 lstm_2 (LSTM)               (None, 32)                42624     
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,773,357
Trainable params: 42,657
Non-trainable params: 4,730,700
_________________________________________________________________
87/87 [==============================] - 2s 12ms/step - loss: 0.6920 - accuracy: 0.5260
Accuracy of the model on Training Data is -  52.595531940460205
36/36 [==============================] - 0s 12ms/step - loss: 0.6928 - accuracy: 0.5188
Accuracy of the model on Testing Data is -  51.87773108482361
E